## Black-Scholes model and greeks

The Black Scholes model considers a geometric brownian motion (GBM) 
$$ \frac{dS} S = \mu dt + \sigma dWt $$
where $W$ is a standard Brownian Motion (SBM). 

The objective is to give a pricing formula for an asset that follows this derivative equation. 

By Itô's formula :
$$V(t, S_t) = v(0, S_0) + \int_0^t \frac{\partial V}{\partial t}(u,  S_u ) du+ \int_0^t \frac{\partial V}{\partial x}(u,  S_u ) dSu +  \frac 1 2\int_0^t \frac{\partial^2 V}{\partial x^2}(u,  S_u ) d\langle 
S \rangle_u $$

Since $S$ is a GBM, the equation can be more simply written $d\langle S \rangle_u = \sigma^2 S^2_u$.
Hence the equation can be written

\begin{align}
V(t, S_t) &= V(0, S_0) + \int_0^t \frac{\partial V}{\partial t}(u,  S_u ) du+ \int_0^t \frac{\partial V}{\partial x}(u,  S_u ) \left( S_u \mu du + \sigma S_u dW_u \right) +  \frac 1 2\int_0^t \frac{\partial^2 V}{\partial x^2}(u,  S_u )\sigma^2 S_u^2\ du\\
&= V(0, S_0) + \int_0^t \frac{\partial V}{ \partial t}(u,  S_u )+ \mu S_u \frac{\partial V}{\partial x}(u,  S_u ) du + \int_0^t \frac{\partial V}{\partial x}(u,  S_u ) \sigma S_u dW_u + \frac 1 2\int_0^t \frac{\partial^2 V}{\partial x^2}(u,  S_u )\sigma^2 S_u^2\ du
\end{align}

In a differential form, this gives : 
$$ \frac{\partial V} {\partial t} = \left(\mu S \frac{\partial V } {\partial S} + \frac{\partial V } {\partial t} + \frac 1 2 \sigma^2 S_u^2\frac{\partial^2 V}{\partial S^2}\right) dt + \sigma S \frac{\partial V } {\partial S} dW $$

The call value in a Black-Scholes model is well known, a proof is reminded here, as well as most of the calculus concerning the Black-Scholes model : https://github.com/myiroslav/Simulation_exercises/blob/main/Simulation_calls.ipynb
 \begin{align}
 C(K, T, r,  \sigma, S_0) &= e^{-rT}  E  \left( S_T \times 1_{\{S_T > K\}} \right) - K e^{-rT} P (S_T > K)\\
 &= N(d_1) S_t - K e^{-r(T-t)} N(d_2) 
 \end{align}
 where 
\begin{align}
d_1 &= \frac 1 {\sigma \sqrt{T-t}} \left( \ln \frac {S_t} K + \left( r + \frac{\sigma^2} 2 \right) (T-t)\right) \\
d_2 &= \frac 1 {\sigma \sqrt{T-t}} \left( \ln \frac {S_t} K + \left( r - \frac{\sigma^2} 2 \right) (T-t)\right) \\
&= d_1 - \sigma \sqrt{T-t}
\end{align}
And $N$ is the CDF of the standard gaussian. 



In [3]:
#basic fucntions for the Black-Scholes model

import numpy as np
from scipy.integrate import quad
import math
import pandas as pd
from scipy import stats
from scipy.stats import norm

def dplus(sigma, T, S0, r, K, t = 0):
  """The d+"""
  return (np.log(S0*np.exp(r*(T-t))/K)/(sigma*np.sqrt(T-t))) + sigma*np.sqrt(T-t)/2

def dminus(sigma, T, S0, r, K, t=0):
  """The d-"""
  return (np.log(S0*np.exp(r*(T-t))/K)/(sigma*np.sqrt(T-t))) - sigma*np.sqrt(T-t)/2

def CallBS(T, S0, K, sigma, r, t = 0):
  """Call BS"""
  d_p = dplus(sigma, T, S0, r, K, t=t)
  d_m = dminus(sigma, T, S0, r, K, t=t)
  return S0*norm.pdf(d_p) - K*np.exp(-r*T)*norm.pdf(d_m)

def PutBS(T, S0, K, sigma, r, t=0):
  """Put BS"""
  d_p = dplus(sigma, T, S0, r, K, t=t)
  d_m = dminus(sigma, T, S0, r, K, t=t)
  return K*np.exp(-r*T)*norm.pdf(-d_m) - S0*norm.pdf(-d_p)

One important thing for contract is the hedging : how do I allocate my funds to cover the risks the best I can ? 
In theory, there is a continuous hedging strategy. However, in practice, one can only hedge its portfolio a certain number of time, resulting in some risk covering errors. 

The greeks are usefool tools to hedge a portfolio. In the Black-Scholes model, it is  possible to have closed formulas :

\begin{align}
\Delta_C &= \frac{\partial V} {\partial S} \\
&= N(d_1)\\
\Delta_P &= N(d_1) - 1\\
\Gamma_C &= \frac{\partial \Delta }{\partial S}\\
 &= \frac 1 {S_t} \frac 1 {\sigma\sqrt{T-t}} N'(d_1)\\
 \Gamma_P &=  \Gamma_C \\
 Vega_C &= \frac{\partial V}{\partial \sigma}\\
 &= S_t \sqrt{T-t} N'(d_1) \\ 
 Vega_P &= Vega_C \\
 \Theta_C &= \frac{\partial V}{\partial T} \\
  &= \frac{S_0 \sigma} {2\sqrt{T-t}}N'(d_1) - K \exp\left(-r(T-t)\right)N(d_2)\\
  \Theta_P &= \frac{\partial V}{\partial T} \\
  &= \frac{S_0 \sigma} {2\sqrt{T-t}}N'(d_1) - K \exp\left(-r(T-t)\right)\left( 1 - N(d_2)\right)\\
  \rho_C &= \frac{\partial V }{\partial r} \\
  &= (T-t) K \exp(-r(T-t)) N(d_2)
  \rho_P &= - (T-t) K \exp(-r(T-t)) \left(1-N(d_2)\right)
\end{align}

In [8]:
def Delta_CallBS(sigma, T, S0, r, K, t=0):
  """Delta Call BS"""
  d_p = dplus(sigma, T, S0, r, K, t=t)
  return norm.cdf(d_p)

def Delta_PutBS(sigma, T, S0, r, K, t=0):
  """Delta Put BS"""
  d_p = dplus(sigma, T, S0, r, K, t=t)
  return norm.cdf(d_p) - 1

def GammaBS(sigma, T, S0, r, K, t=0):
  """Gamma BS"""
  d_p = dplus(sigma, T, S0, r, K, t=t)
  return norm.pdf(d_p) / (S0*sigma*np.sqrt(T-t))

def Vega(sigma, T, S0, r, K, t=0):
  """Vega BS"""
  d_p = dplus(sigma, T, S0, r, K, t=t)
  return norm.pdf(d_p) * (S0*np.sqrt(T-t))

def Theta_CallBS(sigma, T, S0, r, K, t=0):
  """Theta Call BS"""
  d_p = dplus(sigma, T, S0, r, K, t=t)
  d_m = dminus(sigma, T, S0, r, K, t=t)

  a = (S0*sigma) / (2*np.sqrt(T-t))
  b = r*K*np.exp(-r*(T-t))

  return a*norm.pdf(d_p) + b*norm.cdf(d_m)

def Theta_PutBS(sigma, T, S0, r, K, t=0):
  """Theta Put BS"""
  d_p = dplus(sigma, T, S0, r, K, t=t)
  d_m = dminus(sigma, T, S0, r, K, t=t)

  a = (S0*sigma) / (2*np.sqrt(T-t))
  b = r*K*np.exp(-r*(T-t))

  return a*norm.pdf(d_p) - b*(1-norm.cdf(d_m))

def Rho_CallBS(sigma, T, S0, r, K, t=0):
  """Rho Call BS"""
  d_m = dminus(sigma, T, S0, r, K, t=t)
  return (T-t)*K*np.exp(-r*(T-t))*norm.cdf(d_m)

def Rho_PutBS(sigma, T, S0, r, K, t=0):
  """Rho Put BS"""
  d_m = dminus(sigma, T, S0, r, K, t=t)
  return -(T-t)*K*np.exp(-r*(T-t))*(1-norm.cdf(d_m))
def DeltaOfStrike(sigma, T, S0, r, K, t=0):
  """Variation of Call in respect to strike"""
  d_m = dminus(sigma, T, S0, r, K, t=t)
  return -np.exp(-r*T)*norm.cdf(d_m)

In [23]:
#sanitary check with known values (given in class)
S0, r , T, sigma = 100, 0.02, 0.5, 0.25
K = [95, 100, 105]
for k in K :
  print ('Delta = {:.3f}'.format(Delta_CallBS(sigma, T, S0, r, k)),'Delta_Strike = {:.3f}'.format(DeltaOfStrike(sigma, T, S0, r, k)), 'Gamma = {:.3f}'.format(GammaBS(sigma, T, S0, r, k)),'Vega = {:.3f}'.format(Vega(sigma, T, S0, r, k)) ,'Theta = {:.3f}'.format(Theta_CallBS(sigma, T, S0, r, k)), 'rho = {:.3f}'.format(Rho_CallBS(sigma, T, S0, r, k)) , sep = '\t')

Delta = 0.668	Delta_Strike = -0.596	Gamma = 0.021	Vega = 25.662	Theta = 7.548	rho = 28.307
Delta = 0.558	Delta_Strike = -0.482	Gamma = 0.022	Vega = 27.915	Theta = 7.944	rho = 24.123
Delta = 0.448	Delta_Strike = -0.375	Gamma = 0.022	Vega = 27.968	Theta = 7.780	rho = 19.705
